In [ ]:

# III. GRADIENT DESCENT (BATCH GD) - REGULARIZED VERSIONS


# Helper function to calculate Mean Squared Error
def calculate_mse(y_true, y_pred):
    return np.mean((y_pred - y_true)**2)

def ridge_gradient_descent(X, y, lambda_reg, learning_rate=0.01, n_iterations=5000):
    m, n = X.shape
    # Initialize theta (coefficients)
    theta = np.zeros((n, 1))

    for _ in range(n_iterations):
        predictions = X @ theta
        error = predictions - y
        
        # Standard Gradient (1/m * X.T @ error)
        gradients = (1/m) * X.T @ error

        # L2 Regularization Penalty Gradient (2*lambda*w)
        # We create a penalty vector: weights * 2 * lambda_reg
        # Note: Do NOT penalize the intercept/bias term (index 0)
        l2_penalty_gradient = (2 * lambda_reg / m) * theta
        l2_penalty_gradient[0] = 0 # Set penalty for bias term to zero

        # Update theta with the combined gradient
        theta = theta - learning_rate * (gradients + l2_penalty_gradient)
        
    return theta





In [ ]:
#lasso regression
def lasso_gradient_descent(X, y, lambda_reg, learning_rate=0.01, n_iterations=5000):
    m, n = X.shape
    theta = np.zeros((n, 1))

    for _ in range(n_iterations):
        predictions = X @ theta
        error = predictions - y
        
        # Standard Gradient (1/m * X.T @ error)
        gradients = (1/m) * X.T @ error

        # L1 Regularization Penalty Gradient (lambda * sign(w))
        # We create a penalty vector: lambda * sign(weights)
        # Note: Do NOT penalize the intercept/bias term (index 0)
        l1_penalty_gradient = (lambda_reg / m) * np.sign(theta)
        l1_penalty_gradient[0] = 0 # Set penalty for bias term to zero
        
        # Update theta with the combined gradient
        theta = theta - learning_rate * (gradients + l1_penalty_gradient)
        
    return theta


In [ ]:
import matplotlib.pyplot as plt

# =========================================================
# V. REGULARIZATION TUNING & PLOTTING (Manual Implementation)
# =========================================================

print("\n" + "="*60)
print("  REGULARIZATION TUNING (Ridge and Lasso)")
print("="*60)

# Define a range of lambda values on a log scale for tuning
lambda_values = np.logspace(-4, 4, 30) # 30 points from 10^-4 to 10^4

# Set training parameters
# Note: You may need to tune learning_rate or n_iterations if models don't converge
learning_rate = 0.01
n_iterations = 5000

ridge_validation_errors = []
lasso_validation_errors = []

# --- 1. RIDGE REGRESSION TUNING ---
for lambda_val in lambda_values:
    # Train the model on the Training Set
    ridge_weights = ridge_gradient_descent(
        X_train_b_scaled, T_train_col, 
        lambda_reg=lambda_val,
        learning_rate=learning_rate,
        n_iterations=n_iterations
    )
    
    # Predict and evaluate on the Validation Set
    T_validation_predict_ridge = X_validation_b_scaled @ ridge_weights
    mse_val = calculate_mse(T_validation_col, T_validation_predict_ridge)
    ridge_validation_errors.append(mse_val)

# --- 2. LASSO REGRESSION TUNING ---
for lambda_val in lambda_values:
    # Train the model on the Training Set
    lasso_weights = lasso_gradient_descent(
        X_train_b_scaled, T_train_col, 
        lambda_reg=lambda_val,
        learning_rate=learning_rate,
        n_iterations=n_iterations
    )
    
    # Predict and evaluate on the Validation Set
    T_validation_predict_lasso = X_validation_b_scaled @ lasso_weights
    mse_val = calculate_mse(T_validation_col, T_validation_predict_lasso)
    lasso_validation_errors.append(mse_val)

# Find optimal lambda
optimal_lambda_ridge = lambda_values[np.argmin(ridge_validation_errors)]
optimal_lambda_lasso = lambda_values[np.argmin(lasso_validation_errors)]

print(f"Optimal Lambda for Ridge: {optimal_lambda_ridge:.4e} (Min Validation MSE: {np.min(ridge_validation_errors):,.2f})")
print(f"Optimal Lambda for Lasso: {optimal_lambda_lasso:.4e} (Min Validation MSE: {np.min(lasso_validation_errors):,.2f})")